In [1]:
import pandas as pd
import numpy as np
import pickle
from datetime import datetime

In [4]:
# Load the Model
model = pickle.load(open(r'D:\Bike_demand_prediction\xgboost_regressor_r3_0_93_v1.pkl','rb'))

In [5]:
# Load Standard Scaler
sc = pickle.load(open(r'D:\Bike_demand_prediction\StandardScaler.pkl','rb'))

# Convert User input into Model Consumable Format.

In [6]:
# input
date="10/06/2023"
hour=21
temperature=25
humidity=55
wind_speed=67
visibility=25
solar_radiation=0.0
rainfall=0.7
snowfall=0.2
seasons="Winter"
holiday="No Holiday"
functioning_day="Yes"

## Concept Clearence

In [7]:
# first convert holiday and functioning_day
holiday_opt = {'No Holiday':0,'Holiday':1}
functioning_day_opt = {'Yes':1,'No':0}

holiday_choice = holiday_opt[holiday]
functioning_day_choice = functioning_day_opt[functioning_day]
print(holiday_choice, functioning_day_choice)

0 1


In [8]:
# Now encoding the date
from datetime import datetime
dt = datetime.strptime(date,"%d/%m/%Y")
day  = dt.day
month = dt.month
year = dt.year
day_name = dt.strftime('%A')
print(day,month,year,day_name)

10 6 2023 Saturday


In [9]:
# Dataframe :1 ---> other_df
# feature sequence need to pass to model : take sequence from X.columns from main file

# Feature sequence to be pass to model
feature_squence_list = ['Hour', 'Temperature(°C)', 'Humidity(%)', 'Wind speed (m/s)','Visibility (10m)',
                        'Solar Radiation (MJ/m2)', 'Rainfall(mm)','Snowfall (cm)', 'Holiday',
                        'Functioning Day', 'Day', 'Month', 'Year']

# user values to be passed in the sequence of above.
user_input = [hour,temperature,humidity,wind_speed,visibility,solar_radiation,rainfall,snowfall,
               holiday_opt[holiday],functioning_day_opt[functioning_day],dt.day,dt.month,dt.year]
other_df = pd.DataFrame(columns=feature_squence_list,data=[user_input])
other_df

,Hour,Temperature(°C),Humidity(%),Wind speed (m/s),Visibility (10m),Solar Radiation (MJ/m2),Rainfall(mm),Snowfall (cm),Holiday,Functioning Day,Day,Month,Year
0,21,25,55,67,25,0.0,0.7,0.2,0,1,10,6,2023


In [10]:
# Dataframe:2-----> seasons_df
# Now handle the Seasons
seasons_col = ['Spring', 'Summer', 'Winter']
saeasons_data = np.zeros((1,len(seasons_col)),dtype=int)

seasons_df = pd.DataFrame(columns=seasons_col,data=saeasons_data)

if seasons in seasons_col:
  seasons_df[seasons] = 1

seasons_df

,Spring,Summer,Winter
0,0,0,1


In [11]:
# Dataframe: 3 ---> week_day_df
# Now handle the weekday..
week_day_col = ['Monday', 'Saturday', 'Sunday','Thursday', 'Tuesday', 'Wednesday']
week_day_data = np.zeros((1,len(week_day_col)),dtype=int)

week_day_df = pd.DataFrame(columns=week_day_col,data=week_day_data)

if day_name in week_day_col:
  week_day_df[day_name] = 1

week_day_df

,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
0,0,1,0,0,0,0


In [12]:
# Concate the 3 dataFrame and make it 1 DataFrame for model input
test_df = pd.concat([other_df,seasons_df,week_day_df],axis=1)
test_df

,Hour,Temperature(°C),Humidity(%),Wind speed (m/s),Visibility (10m),Solar Radiation (MJ/m2),Rainfall(mm),Snowfall (cm),Holiday,Functioning Day,...,Year,Spring,Summer,Winter,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
0,21,25,55,67,25,0.0,0.7,0.2,0,1,...,2023,0,0,1,0,1,0,0,0,0


In [13]:
# scalling the data
test_df_sc = sc.transform(test_df)
test_df_sc

array([[ 1.36681471,  1.01214045, -0.15409119, 63.11301355, -2.33371591,
        -0.65681529,  0.46358031,  0.29083855, -0.22754885,  0.18254726,
        -0.64636159, -0.14853965, 18.24263884, -0.57427456, -0.58152447,
         1.72810536, -0.39840954,  2.45255275, -0.41815465, -0.40773843,
        -0.40559398, -0.41815465]])

In [14]:
# prediction
pred_test =model.predict(test_df_sc)
print(f"Rented Bike Demand on Date : {date} & Time : {hour} is : {int(pred_test[0].round())}")

Rented Bike Demand on Date : 10/06/2023 & Time : 21 is : 587


## Implement Through Function

In [ ]:
def user_input_to_model_input(user_input):

  # input :
  # {'date':'10/06/2023','hour':21,'temperature':25',humidity':55,'wind_speed':67,'visibility':25,
  # 'solar_radiation':0.0,'rainfall':0.7,'snowfall':0.2,'seasons':"Winter",'holiday':"No Holiday",'functioning_day':"Yes"}
  input = [i for i in user_input.values()]
  #-----------------------------------------------------------------------------------------------------------------------
  # first convert holiday and functioning_day
  holiday_opt = {'No Holiday':0,'Holiday':1}
  functioning_day_opt = {'Yes':1,'No':0}

  holiday_choice = holiday_opt[input[10]]
  functioning_day_choice = functioning_day_opt[input[11]]

  #-----------------------------------------------------------------------------------------------------------------------
  # Now encoding the date
  dt = datetime.strptime(input[0],"%d/%m/%Y")
  day  = dt.day
  month = dt.month
  year = dt.year
  day_name = dt.strftime('%A')

  #-----------------------------------------------------------------------------------------------------------------------
  # Dataframe :1 ---> other_df
  # Feature sequence to be pass to model
  feature_squence_list = ['Hour', 'Temperature(°C)', 'Humidity(%)', 'Wind speed (m/s)','Visibility (10m)',
                          'Solar Radiation (MJ/m2)', 'Rainfall(mm)','Snowfall (cm)', 'Holiday',
                          'Functioning Day', 'Day', 'Month', 'Year']

  # user values to be passed in the sequence of above.
  user_input =input[1:9]
  other_input =[holiday_choice,functioning_day_choice,day,month,year]
  user_input.extend(other_input)

  other_df = pd.DataFrame(columns=feature_squence_list,data=[user_input])

  #-----------------------------------------------------------------------------------------------------------------------
  # Dataframe:2-----> seasons_df
  # Now handle the Seasons
  seasons_col = ['Spring', 'Summer', 'Winter']
  saeasons_data = np.zeros((1,len(seasons_col)),dtype=int)

  seasons_df = pd.DataFrame(columns=seasons_col,data=saeasons_data)

  if input[9] in seasons_col:    # input[9] --> 'Winter'
    seasons_df[input[9]] = 1

  #-----------------------------------------------------------------------------------------------------------------------
  # Dataframe: 3 ---> week_day_df
  # Now handle the weekday..
  week_day_col = ['Monday', 'Saturday', 'Sunday','Thursday', 'Tuesday', 'Wednesday']
  week_day_data = np.zeros((1,len(week_day_col)),dtype=int)

  week_day_df = pd.DataFrame(columns=week_day_col,data=week_day_data)

  if day_name in week_day_col:
    week_day_df[day_name] = 1

  #-----------------------------------------------------------------------------------------------------------------------
  # Concate the 3 dataFrame and make it 1 DataFrame for model input
  test_df = pd.concat([other_df,seasons_df,week_day_df],axis=1)
  test_df

  #-----------------------------------------------------------------------------------------------------------------------
  # scalling the data
  test_df_sc = sc.transform(test_df)
  return test_df_sc

In [ ]:
test ={'date':'05/06/2024','hour':18,'temperature':-10,'humidity':35,'wind_speed':100,'visibility':20,'solar_radiation':0.0,
       'rainfall':0.4,'snowfall':0.7,'seasons':"Winter",'holiday':"No Holiday",'functioning_day':"Yes"}
test_data = user_input_to_model_input(test)

In [ ]:
# test_data :now it is in final format for provide to model
test_data

In [ ]:
# prediction
pred_test =model.predict(test_data)
if pred_test[0] <= 0:
  print(f"Rented Bike Demand on Date : {test['date']} & Time : {test['hour']} is : 0 ")
else:
  print(f"Rented Bike Demand on Date : {test['date']} & Time : {test['hour']} is : {int(pred_test[0].round())}")

# At the end our Model is ready for deployment....

Date (dd/mm/yyyy):10/06/2023
Hour (0-23):21
Temperature in C:25
Humidity :55
Wind Speed :67
Visibility :25
Solar_radiation :0
Rainfall :0.7
Snowfall :0.2
Seasons (Antum/Spring/Summer/Winter):Winter
Holiday (No Holiday/Holiday):No Holiday
Functioning_day (Yes/No) :Yes